In [1]:
import pandas as pd 
import datetime as dt 
import os 
from wmf import wmf
from multiprocessing import Pool
import numpy as np
import pickle 
import alarmas as al
import glob
import time
import pylab as pl

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### 1. Se crea el .nc de la cuenca para simulacion:
Esto se hace copiando y siguiendo lo realizado en una de las plantillas alojadas en:

    '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/jupyter_codes/Preparacuencas'

Para respetar la estructura y mantener la organización, se debe nombrar el proyecto de la cuenca y todos los archivos a crear dentro de ella teniendo en cuenta:
 
 - Nombre de la cuenca
 - Identifación del punto esocgido para trazar, en el mejor de los casos el código de una estación nivel ubicada en el mismo, en el peor el nombre del sector.
 
### 2. Se prueban las calibraciones y se crean los binarios de storage considerados para hacer updates:
Siguiendo las plantilla en:
 
     '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/jupyter_codes/Calibraciones'
Los binarios de storage se crean de acuerdo a lo estipulado en el cofigfile para actualizar las CI.

### 2. Se crea el lugar donde alojar el proyecto 

A partir de la estructura del de muestra alojado en:
    
    '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/'

Que pertenece al repo actual Alarmas.

Desde esta plantilla se crean las divisiones dentro de la carpeta, los archivos: **configfile.md**, **Cron_Ejecucion.py** y los **README.md** se copian desde el proyecto muestra y se editan conforme se necesite.

### 3. Se crean los archivos de lluvia.

# -------------------------------------------- Desarrollo ---------------------------------------------

### Se crea el lugar donde alojar el proyecto:

- Ruta actual del repo: 

In [3]:
ruta_repo='/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/'

In [7]:
#INPUT.
nombreproyecto='Op_AMVA60m'

# Se crea la carpeta del proeyecto
comando='mkdir '+ruta_repo+nombreproyecto
os.system(comando)

- Estructura del repo

#carpeta de resultados
ruta_in1=ruta_repo+nombreproyecto+'/results'
comando='mkdir '+ruta_in1
os.system(comando)
#carpeta de backsto donde se alojaran los
ruta_in2=ruta_repo+nombreproyecto+'/backsto'
comando='mkdir '+ruta_in2
os.system(comando)

#rutas en 1
ruta_in11=ruta_in1+'/01_rain'
comando='mkdir '+ruta_in11
os.system(comando)

ruta_in12=ruta_in1+'/02_caudalsim'
comando='mkdir '+ruta_in12
os.system(comando)

ruta_in13=ruta_in1+'/03_humedadsim'
comando='mkdir '+ruta_in13
os.system(comando)

ruta_in14=ruta_in1+'/04_slides'
comando='mkdir '+ruta_in14
os.system(comando)

Nota: La estructura de carpetas del nuevo lugar del proyecto debe quedar igual al proyecto de muestra.

# .

De ahora en adelante el lugar que aloja el proyecto se referirá como: ../

En este punto, y antes de continuar, es necesario copiar y dejar listos en la carpeta del proyecto:

 - los archivos binarios de storage en ../backsto
 - configfile.md y Cron_Ejecucion.py en ../

### Se carga el configfile de la cuenca para leer las rutas

In [2]:
ruta_configuracion_1 = '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/configfile.md'
RutasList = al.get_rutesList(ruta_configuracion_1)

# Lee rutas de objetos de entrada
ruta_cuenca = al.get_ruta(RutasList, 'ruta_cuenca')
ruta_campos = al.get_ruta(RutasList, 'ruta_campos')
# Lee rutas de salida - lluvia radar
ruta_out_rain = al.get_ruta(RutasList, 'ruta_rain')

### Se deben crear un binario de lluvia historico para que el cron actualice: lluvia_historica.bin - .hdr
 Para esto se ejecuta:

ruta_codigo='/home/nicolas/self_code/RadarConvStra2Basin3.py'
fechaI='2018-01-11'
fechaF='2018-01-12'
rutaRes=ruta_out_rain+'lluvia_historica'
hora1='00:00:00'
hora2='23:59:00'
comando=ruta_codigo+' '+fechaI+' '+fechaF+' '+ruta_cuenca+' '+ruta_campos+' '+rutaRes+' -v -1 '+hora1+' -2 '+hora2
os.system(comando)

# .!!
En este punto se debe ensayar el Cron_Ejecución hasta la parte de la lluvia, y a partir de esta una ejecución de modelo para crear archivos Qsim.json

### Se crean los archivos donde se alojan los historicos de simulacion: Qhist y Shits.
A partir de un archivo Qsim.json y Ssim.StOhdr (.csv), cuyas rutas se definen en el configfile, se ejectutan las sgte. funcion de alarmas.py:

**Nota:** Esta función usa df.reindex(freq='5min'), esta sólo funciona si el index.freq es exactamente el indicado, sino pone todo en nan.
- Mientras se hagan pruebas se debe probar esta función con resultados corridos en el mismo minuto con que se escriben los historicos (en un rango de 1 a 5 min.), e.g. si el index inicial del hist tiene minuto 02, las corridas de prueba debe ser el minuto 02 o 07.

- Cuando se vaya a poner operacional el modelo, se debe escribir nuevamente los histfile con datos en minuto 05 para que siempre funcione el df.reindex()

In [3]:
al.model_write_rutesHists(ruta_configuracion_1)

Aviso: El archivo Qhist: Qsim_001hist.json ya existe, desea sobre-escribirlo, perdera la historia de este!! (S o N): S
Aviso: Se crean los Qhist, el archivo Qsim usado para crear las rutas es: /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/02_caudalsim/Qsim_001.json
Aviso: Se crean los Shist , el archivo Ssim usado para crear las rutas es: /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/03_humedadsim/storage_001.StOhdr


# .!!
En este punto se debe ensayar el Cron_Ejecución hasta la parte de la función Graph_Levels() que necesita un Qhist con una hora dehistoria para poder graficar.